In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors

mol = Chem.MolFromSmiles(smiles)
fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
desc = [Descriptors.MolWt(mol), Descriptors.NumHDonors(mol)]


NameError: name 'smiles' is not defined

In [4]:
import pandas as pd

df = pd.read_csv("gdsc_pancreas_flexiscatter.csv")

# Split combination column
df[["drug1", "drug2"]] = df["Combination"].str.split(" \+ ", expand=True)

# Select columns and rename
comb_data = df[["drug1", "drug2", "Cell line name", "Bliss Emax"]].rename(
    columns={"Cell line name": "cell_line", "Bliss Emax": "synergy_bliss"}
)

# Save as TSV for MatchMaker
comb_data.to_csv("comb_data.tsv", sep="\t", index=False)


<>:6: SyntaxWarning: invalid escape sequence '\+'
<>:6: SyntaxWarning: invalid escape sequence '\+'
C:\Users\ardat\AppData\Local\Temp\ipykernel_30820\4101545813.py:6: SyntaxWarning: invalid escape sequence '\+'
  df[["drug1", "drug2"]] = df["Combination"].str.split(" \+ ", expand=True)


In [6]:
import pandas as pd

path = "rnaseq_all_20250117.csv"

# Just peek at the first few rows to see the column names
df_head = pd.read_csv(path, nrows=5)
print(df_head.columns)
df_head


Index(['model_name', 'model_id', 'gene_symbol', 'ensembl_gene_id', 'gene_id',
       'htseq_read_count', 'rsem_expected_count', 'rsem_fpkm', 'rsem_tpm',
       'htseq_fpkm', 'data_source'],
      dtype='object')


,model_name,model_id,gene_symbol,ensembl_gene_id,gene_id,htseq_read_count,rsem_expected_count,rsem_fpkm,rsem_tpm,htseq_fpkm,data_source
0,LC-1-sq-SF,SIDM01226,TSPAN6,ENSG00000000003,SIDG39683,NaN,2383.0,NaN,4.3320,NaN,Broad
1,COG-AR-359,SIDM01488,TSPAN6,ENSG00000000003,SIDG39683,NaN,2529.0,NaN,4.5674,NaN,Broad
2,COLO-794,SIDM00514,TSPAN6,ENSG00000000003,SIDG39683,NaN,1552.0,NaN,3.1506,NaN,Broad
3,KKU-M213,SIDM01801,TSPAN6,ENSG00000000003,SIDG39683,NaN,5657.0,NaN,5.0853,NaN,Broad
4,RT4,SIDM01085,TSPAN6,ENSG00000000003,SIDG39683,NaN,22806.0,NaN,6.7294,NaN,Broad


In [7]:
import pandas as pd

path = "rnaseq_all_20250117.csv"
out_path = "rnaseq_pdac_subset.csv"

pdac_lines = [
    "PL4","SUIT-2","YAPC","MZ1-PC","DAN-G","Hs-766T","KP-1N","KP-3","KP-4",
    "BxPC-3","PANC-02-03","PANC-04-03","PANC-10-05","PANC-03-27","CAPAN-1",
    "CFPAC-1","SW1990","MIA-PaCa-2","PSN1","HuP-T3","CAPAN-2","HuP-T4",
    "KP-2","PANC-08-13","AsPC-1","PA-TU-8988T","SU8686","PA-TU-8902","HPAC"
]

chunksize = 100_000  # adjust if needed
first_chunk = True

for chunk in pd.read_csv(path, chunksize=chunksize):
    # ⚠️ if the column name is different, change "model_name" here
    mask = chunk["model_name"].isin(pdac_lines)
    sub = chunk[mask]
    if not sub.empty:
        sub.to_csv(out_path,
                   mode="w" if first_chunk else "a",
                   index=False,
                   header=first_chunk)
        first_chunk = False

print("Done. Filtered PDAC subset saved to:", out_path)


Done. Filtered PDAC subset saved to: rnaseq_pdac_subset.csv


In [2]:
import pandas as pd
import numpy as np

subset_path = r"C:\Users\ardat\OneDrive - Sabancı Üniversitesi\Desktop\bitirme\matchmaker\rnaseq_pdac_subset.csv"
subset = pd.read_csv(subset_path)

print(subset.shape)
subset.head()


(1160291, 11)


,model_name,model_id,gene_symbol,ensembl_gene_id,gene_id,htseq_read_count,rsem_expected_count,rsem_fpkm,rsem_tpm,htseq_fpkm,data_source
0,CFPAC-1,SIDM00130,TSPAN6,ENSG00000000003,SIDG39683,NaN,3857.0,NaN,4.6496,NaN,Broad
1,AsPC-1,SIDM00899,TSPAN6,ENSG00000000003,SIDG39683,NaN,5798.0,NaN,5.3837,NaN,Broad
2,HuP-T4,SIDM00531,TSPAN6,ENSG00000000003,SIDG39683,NaN,5230.0,NaN,4.4718,NaN,Broad
3,CAPAN-1,SIDM00934,TSPAN6,ENSG00000000003,SIDG39683,NaN,2397.0,NaN,3.8410,NaN,Broad
4,KP-3,SIDM00571,TSPAN6,ENSG00000000003,SIDG39683,NaN,2398.0,NaN,4.3406,NaN,Broad


In [3]:
# Keep only what we need
sub = subset[["model_name", "gene_symbol", "rsem_tpm"]].copy()

# Drop missing expression values
sub = sub.dropna(subset=["rsem_tpm"])

# If the same (model_name, gene_symbol) appears multiple times, average them
sub = sub.groupby(["model_name", "gene_symbol"], as_index=False)["rsem_tpm"].mean()

# Pivot: rows = cell line, columns = gene, values = expression
expr_wide = sub.pivot(
    index="model_name",
    columns="gene_symbol",
    values="rsem_tpm"
).fillna(0.0)

print("Expression matrix shape:", expr_wide.shape)
expr_wide.head()


Expression matrix shape: (29, 41154)


gene_symbol,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A2ML1-AS2,A2MP1,A3GALT2,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZYXP1,ZZEF1,ZZZ3
model_name,,,,,,,,,,,,,,,,,,,,,
AsPC-1,0.1375,0.1890,3.4529,0.0704,0.0000,0.0144,0.0,0.0,0.0000,0.0286,...,0.7225,1.7355,3.1010,0.0000,0.0,2.0286,5.9878,0.0,3.2735,3.7038
BxPC-3,0.0000,0.0000,0.0000,0.2141,0.0144,1.8875,0.0,0.0,0.0000,0.0000,...,1.2141,2.7442,3.5969,0.0286,0.0,2.2630,7.0491,0.0,3.1309,3.7655
CAPAN-1,0.3785,0.3561,0.3219,0.0566,0.2265,0.2141,0.0,0.0,0.0286,0.0000,...,0.8156,1.6229,3.8797,0.0286,0.0,2.0144,5.3161,0.0,3.0036,3.8649
CAPAN-2,0.2750,0.2265,0.0000,0.1110,0.4854,2.2957,0.0,0.0,0.0000,0.0000,...,1.2327,2.1110,3.4777,0.0566,0.0,1.4906,5.9258,0.0,2.8895,3.2141
CFPAC-1,0.1506,0.0286,0.0841,0.2016,0.3561,0.0144,0.0,0.0,0.0000,0.0000,...,0.8875,2.0738,3.6029,0.2750,0.0,2.3561,5.7695,0.0,2.3618,3.9155


In [5]:
import pandas as pd
import numpy as np

subset = pd.read_csv(r"C:\Users\ardat\OneDrive - Sabancı Üniversitesi\Desktop\bitirme\matchmaker\rnaseq_pdac_subset.csv")
sub = subset[["model_name","gene_symbol","rsem_tpm"]].dropna()

# average duplicates
sub = sub.groupby(["model_name","gene_symbol"],as_index=False)["rsem_tpm"].mean()

# ordered list of genes (shared index)
genes = sorted(sub["gene_symbol"].unique())

# prepare output file
out_path = r"C:\Users\ardat\OneDrive - Sabancı Üniversitesi\Desktop\bitirme\matchmaker\cell_line_gex.csv"
open(out_path, "w").close()   # clear file

# process one cell line at a time
for cl, df_cl in sub.groupby("model_name"):
    vec = (
        df_cl.set_index("gene_symbol")
             .reindex(genes)["rsem_tpm"]
             .fillna(0)
             .astype(np.float32)
             .values
    )
    pd.DataFrame([vec]).to_csv(out_path, mode="a", header=False, index=False)
    print("✔ wrote", cl)


✔ wrote AsPC-1
✔ wrote BxPC-3
✔ wrote CAPAN-1
✔ wrote CAPAN-2
✔ wrote CFPAC-1
✔ wrote DAN-G
✔ wrote HPAC
✔ wrote Hs-766T
✔ wrote HuP-T3
✔ wrote HuP-T4
✔ wrote KP-1N
✔ wrote KP-2
✔ wrote KP-3
✔ wrote KP-4
✔ wrote MIA-PaCa-2
✔ wrote MZ1-PC
✔ wrote PA-TU-8902
✔ wrote PA-TU-8988T
✔ wrote PANC-02-03
✔ wrote PANC-03-27
✔ wrote PANC-04-03
✔ wrote PANC-08-13
✔ wrote PANC-10-05
✔ wrote PL4
✔ wrote PSN1
✔ wrote SU8686
✔ wrote SUIT-2
✔ wrote SW1990
✔ wrote YAPC


In [8]:
import pandas as pd
import numpy as np

subset_path = r"C:\Users\ardat\OneDrive - Sabancı Üniversitesi\Desktop\bitirme\matchmaker\rnaseq_pdac_subset.csv"
subset = pd.read_csv(subset_path)

# Keep only what we need
sub = subset[["model_name", "gene_symbol", "rsem_tpm"]].dropna()

# Average duplicates (same cell line + gene)
sub = sub.groupby(["model_name", "gene_symbol"], as_index=False)["rsem_tpm"].mean()

# Pivot: rows = cell line, columns = gene
expr_wide = sub.pivot(
    index="model_name",
    columns="gene_symbol",
    values="rsem_tpm"
).fillna(0.0)

print("expr_wide shape (cell lines x genes):", expr_wide.shape)


expr_wide shape (cell lines x genes): (29, 41154)


In [9]:
# Compute variance of each gene across cell lines
gene_var = expr_wide.var(axis=0)

# Choose how many genes to keep
N_GENES = 2000     # you can try 1000, 2000, 5000, etc.

top_genes = gene_var.sort_values(ascending=False).head(N_GENES).index

expr_reduced = expr_wide[top_genes].astype("float32")  # float32 to save memory
print("expr_reduced shape:", expr_reduced.shape)


expr_reduced shape: (29, 2000)


In [10]:
comb = pd.read_csv(
    r"C:\Users\ardat\OneDrive - Sabancı Üniversitesi\Desktop\bitirme\matchmaker\comb_data.tsv",
    sep="\t"
)

print("comb_data rows:", len(comb))
print("unique cell lines in comb_data:", comb["cell_line"].nunique())
print("expr_reduced index (cell lines):", expr_reduced.index.tolist())


comb_data rows: 66300
unique cell lines in comb_data: 29
expr_reduced index (cell lines): ['AsPC-1', 'BxPC-3', 'CAPAN-1', 'CAPAN-2', 'CFPAC-1', 'DAN-G', 'HPAC', 'Hs-766T', 'HuP-T3', 'HuP-T4', 'KP-1N', 'KP-2', 'KP-3', 'KP-4', 'MIA-PaCa-2', 'MZ1-PC', 'PA-TU-8902', 'PA-TU-8988T', 'PANC-02-03', 'PANC-03-27', 'PANC-04-03', 'PANC-08-13', 'PANC-10-05', 'PL4', 'PSN1', 'SU8686', 'SUIT-2', 'SW1990', 'YAPC']


In [11]:
import os

out_path = r"C:\Users\ardat\OneDrive - Sabancı Üniversitesi\Desktop\bitirme\matchmaker\cell_line_gex.csv"

# Clear if exists
if os.path.exists(out_path):
    os.remove(out_path)

missing = set()

for i, row in comb.iterrows():
    cl = row["cell_line"]
    if cl in expr_reduced.index:
        vec = expr_reduced.loc[cl].values
    else:
        # if some cell line has no expression, fill with zeros
        missing.add(cl)
        vec = np.zeros(expr_reduced.shape[1], dtype="float32")
    
    # Append line to CSV
    pd.DataFrame([vec]).to_csv(
        out_path,
        mode="a",
        header=False,
        index=False
    )

    if (i + 1) % 5000 == 0:
        print(f"Wrote {i+1} / {len(comb)} rows")

if missing:
    print("⚠ Missing expression for these cell lines (filled with zeros):", missing)

print("Done. Final cell_line_gex.csv should have shape:", len(comb), "x", expr_reduced.shape[1])


Wrote 5000 / 66300 rows
Wrote 10000 / 66300 rows
Wrote 15000 / 66300 rows
Wrote 20000 / 66300 rows
Wrote 25000 / 66300 rows
Wrote 30000 / 66300 rows
Wrote 35000 / 66300 rows
Wrote 40000 / 66300 rows
Wrote 45000 / 66300 rows
Wrote 50000 / 66300 rows
Wrote 55000 / 66300 rows
Wrote 60000 / 66300 rows
Wrote 65000 / 66300 rows
Done. Final cell_line_gex.csv should have shape: 66300 x 2000


In [12]:
import pandas as pd

comb = pd.read_csv("comb_data.tsv", sep="\t")
d1   = pd.read_csv("drug1_chemicals.csv", header=None)
d2   = pd.read_csv("drug2_chemicals.csv", header=None)
gex  = pd.read_csv("cell_line_gex.csv", header=None)

print("comb_data:", comb.shape)
print("drug1_chemicals:", d1.shape)
print("drug2_chemicals:", d2.shape)
print("cell_line_gex:", gex.shape)

# All rows must match
assert len(comb) == len(d1) == len(d2) == len(gex)
print("✅ All files have the same number of rows.")




comb_data: (66300, 4)
drug1_chemicals: (66300, 1024)
drug2_chemicals: (66300, 1024)
cell_line_gex: (66300, 2000)
✅ All files have the same number of rows.


In [2]:
import numpy as np
import pandas as pd

comb = pd.read_csv("comb_data.tsv", sep="\t")
N = len(comb)
idx = np.arange(N)

# Randomly shuffle and split 70/15/15
np.random.seed(42)
np.random.shuffle(idx)

train_end = int(0.7 * N)
val_end = int(0.85 * N)

train_idx = idx[:train_end]
val_idx = idx[train_end:val_end]
test_idx = idx[val_end:]

# Save as plain text files (no headers)
np.savetxt("train_inds.txt", train_idx, fmt="%d")
np.savetxt("val_inds.txt", val_idx, fmt="%d")
np.savetxt("test_inds.txt", test_idx, fmt="%d")

print("✅ Created index files:")
print("train_inds:", train_idx.shape)
print("val_inds:", val_idx.shape)
print("test_inds:", test_idx.shape)


✅ Created index files:
train_inds: (46410,)
val_inds: (9945,)
test_inds: (9945,)
